In [1]:
import requests
import time
from datetime import datetime
import dateutil.parser
import os
import yaml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium import plugins



In [3]:
print("hellooo")

hellooo


In [4]:
# pip install folium
# 
!pip install dash

Error processing line 1 of c:\python38\lib\site-packages\matplotlib-3.3.2-py3.8-nspkg.pth:

  Traceback (most recent call last):
    File "c:\python38\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 553, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [5]:
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
SERVER = 'EMSTrack'
cfg = cfg[SERVER]

In [6]:
print("hello")

hello


In [7]:
base_url = cfg['url']

def get (url, params=None, extend=True):
    global base_url, token
    set_token()
    if extend == True:
        url = base_url + url
    headers = {'Content-Type': 'application/json', 'Authorization': f'Token {token}'}

    response = requests.get(url, params=params, headers = headers)
    response.raise_for_status()
    return response

def post (url, params=None, extend=True):
    global base_url, token
    set_token()
    if extend == True:
        url = base_url + url
    headers = {'Content-Type': 'application/json', 'Authorization': f'Token {token}'}

    response = requests.post(url, headers = headers, data=params)
    response.raise_for_status()
    return response


def set_token ():
    global token
    global token_timestamp

    try:
        token_timestamp
    except NameError:
        token_timestamp = 0

    if token_timestamp == 0:
        param = {
            'username' : cfg['username'],
            'password' : cfg['password']
        }
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        url = cfg['authurl']
        res = requests.post(url, data=param)
        res.raise_for_status()
        token_timestamp = time.time()
        token = res.json()['token']



res = get('ambulance')  

# if __name__ == "__main__":
#   main()

In [8]:
ambulances = res.json()

In [9]:
vehicles = {}
df = pd.DataFrame()

# def getTime(start,end): 
#     print("")


for ambulance in ambulances:
    id = ambulance['id']
    data = get(f'ambulance/{id}/updates/?filter=2019-09-01T15:00:00.000Z,2019-10-30T15:00:00.000Z').json()
                    
#     data = get(f'ambulance/{id}/updates/').json()#?filter=2020-10-28T15:00:00.000Z,2020-10-30T15:00:00.000Z').json() 

    data = [{'id':id, 'identifier':ambulance['identifier'], **item} for item in data]
    df = df.append(pd.DataFrame(data))
    display(df)
    #print(data) 
    vehicles[id] = {'ambulance':ambulance, 'data':data}
 
    
     #Just doing for one for now


,id,identifier,location,orientation,status,timestamp,updated_by_username,updated_on
0,1,Unidad de Prueba 1,"{'latitude': 32.4988403, 'longitude': -116.965...",290.686768,AV,2019-09-23T17:34:34.832000Z,controlmedico1,2019-09-23T17:25:10.554488Z
1,1,Unidad de Prueba 1,"{'latitude': 32.4988403, 'longitude': -116.965...",290.686768,UK,2019-09-23T17:34:34.832000Z,cvera,2019-09-23T19:13:11.263483Z
2,1,Unidad de Prueba 1,"{'latitude': 32.4988403, 'longitude': -116.965...",290.686768,OS,2019-09-23T17:34:34.832000Z,cvera,2019-09-23T18:42:38.847581Z
3,1,Unidad de Prueba 1,"{'latitude': 32.4988403, 'longitude': -116.965...",290.686768,UK,2019-09-23T17:17:03.843000Z,controlmedico1,2019-09-23T17:24:26.339625Z
4,1,Unidad de Prueba 1,"{'latitude': 32.4988403, 'longitude': -116.965...",290.686768,AV,2019-09-23T17:17:03.843000Z,controlmedico1,2019-09-23T17:07:29.258502Z
5,1,Unidad de Prueba 1,"{'latitude': 32.4984842, 'longitude': -116.965...",128.721375,AV,2019-09-23T17:16:42.437000Z,controlmedico1,2019-09-23T17:07:08.000820Z
6,1,Unidad de Prueba 1,"{'latitude': 32.4986846, 'longitude': -116.965...",125.573738,AV,2019-09-23T17:16:31.805000Z,controlmedico1,2019-09-23T17:07:07.995988Z
7,1,Unidad de Prueba 1,"{'latitude': 32.4988805, 'longitude': -116.965...",37.765297,AV,2019-09-23T17:00:14.724000Z,controlmedico1,2019-09-23T16:50:56.527687Z
8,1,Unidad de Prueba 1,"{'latitude': 32.4986064, 'longitude': -116.965...",198.336899,AV,2019-09-23T17:00:09.222000Z,controlmedico1,2019-09-23T16:50:34.791888Z
9,1,Unidad de Prueba 1,"{'latitude': 32.4988925, 'longitude': -116.965...",225.988220,AV,2019-09-23T16:41:12.919000Z,controlmedico1,2019-09-23T16:31:54.458765Z


In [10]:
def splitlotlan(row):
    row['lon'] = row['location']['longitude']
    row['lat'] = row['location']['latitude']
    if row['lat'] == 37.4219983 and row['lon'] == -122.084:
        return None
    return row
  
df = df.apply(splitlotlan, axis=1).dropna()


In [11]:
m = folium.Map(
    location=list(df[['lat', 'lon']].mean().values.tolist()),
    zoom_start=16
)
m.fit_bounds(
    [df[['lat', 'lon']].min().values.tolist(), df[['lat', 'lon']].max().values.tolist()]
)
# Lon, Lat order.
lines = []
dat = list(zip(df['lon'],df['lat'],df['timestamp']))
for i in range (1, len(dat)):
    lines.append({
      'coordinates': [
            list(dat[i][:2]),
            list(dat[i-1][:2]),
        ],
        'dates': [
            dat[i][2],
            dat[i-1][2]
        ],
        'color': 'orange'
  })
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
        },
        'properties': {
            'times': line['dates'],
            'style': {
                'color': line['color'],
                'weight': line['weight'] if 'weight' in line else 5
            }
        }
    }
    for line in lines
]

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M', add_last_point=True).add_to(m)
m.save(os.path.join( 'sample.html'))

m

In [14]:
import plotly.graph_objects as go # or plotly.express as px
 # or any Plotly Express function e.g. px.bar(...)
# fig.add_trace( ... )
# fig.update_layout( ... )

import dash
import dash_core_components as dcc
import dash_html_components as html

import plotly.express as px


ModuleNotFoundError: No module named 'dash'

In [13]:

fig = px.line_mapbox(df[df['identifier']=='7BST414'], lat="lat", lon="lon", color="identifier", zoom=3, height=500)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=4, mapbox_center_lat = 41,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()



NameError: name 'px' is not defined

In [18]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\annem\documents\python.exe -m pip install --upgrade pip' command.


In [19]:
data = df
data['timestamp'] = pd.to_datetime(data.timestamp)
data = data.sort_values(by='timestamp', ascending=True).reset_index()
center = {'lon': data['lon'].mean(), 'lat':data['lat'].mean()}
fig = go.Figure()

for step in range(2,data.shape[0],5):
    fig.add_trace(
        go.Scattermapbox(
        visible=False,
        mode = "markers+lines",
        lon = data.loc[:step:5,'lon'],
        lat = data.loc[:step:5,'lat'],
        marker = {'size': 10}))
          
        

        
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': center,
        'style': "stamen-terrain",
        'zoom': 8}
)

fig.data[10].visible = True

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()
fig.write_html("plotly.html")


IndexError: tuple index out of range